In [105]:
import numpy as np
from helpers_trials import *

from implementations_trial import *
from preprocess_trial import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
variables = [
    "GENHLTH", "_RFHLTH",
    "HLTHPLN1", "_HCVU651",
    "BPHIGH4", "_RFHYPE5",
    "BLOODCHO", "CHOLCHK", "_CHOLCHK",
    "BLOODCHO", "TOLDHI2", "_RFCHOL",
    "CVDINFR4", "CVDCRHD4", "_MICHD",
    "ASTHMA3", "_LTASTH1",
    "ASTHMA3", "ASTHNOW", "_CASTHM1",
    "ASTHMA3", "ASTHNOW", "_ASTHMS1",
    "HAVARTH3", "_DRDXAR1",
    "MRACE1", "ORACE3", "MRACASC1", "_PRACE1",
    "MRACE1", "MRACORG1", "MRACASC1", "_MRACE1",
    "HISPANC3", "_HISPANC",
    "MRACE1", "HISPANC3", "MRACORG1", "MRACASC1", "_MRACE1", "_HISPANC", "_RACE",
    "HISPANC3", "MRACE1", "_RACE", "_RACEG21",
    "HISPANC3", "MRACE1", "_RACE", "_RACEGR3",
    "MRACE1", "HISPANC3", "_RACEGR3", "_RACE_G1",
    "AGE", "_AGEG5YR",
    "AGE", "_AGE65YR",
    "AGE", "_IMPAGE", "_AGE80",
    "AGE", "_IMPAGE", "_AGE_G",
    "HEIGHT3", "HTIN4",
    "HEIGHT3", "HTIN4", "HTM4",
    "WEIGHT2", "WTKG3",
    "SEX", "WEIGHT2", "HEIGHT3", "HTIN4", "HTM4", "WTKG3", "_BMI5",
    "SEX", "WEIGHT2", "HEIGHT3", "HTIN4", "HTM4", "WTKG3", "_BMI5", "_BMI5CAT",
    "SEX", "WEIGHT2", "HEIGHT3", "HTIN4", "HTM4", "WTKG3", "_BMI5", "_RFBMI5",
    "CHILDREN", "_CHLDCNT",
    "EDUCA", "_EDUCAG",
    "INCOME2", "_INCOMG",
    "SMOKE100", "SMOKDAY2", "_SMOKER3",
    "SMOKE100", "SMOKDAY2", "_SMOKER3", "_RFSMOK3",
    "ALCDAY5", "DRNKANY5",
    "ALCDAY5", "DROCDY3_",
    "ALCDAY5", "DRNK3GE5", "_RFBING5",
    "ALCDAY5", "AVEDRNK", "DROCDY3_", "_DRNKWEK",
    "SEX", "ALCDAY5", "AVEDRNK", "_DRNKWEK", "_RFDRHV5",
    "FRUITJU1", "FTJUDA1_",
    "FRUIT1", "FRUTDA1_",
    "FVBEANS", "BEANDAY_",
    "FVGREEN", "GRENDAY_",
    "FVORANG", "ORNGDAY_",
    "VEGETAB1", "VEGEDA1_",
    "FRUITJU1", "FRUIT1", "FTJUDA1_", "FRUTDA1_", "_MISFRTN",
    "VEGETAB1", "FVGREEN", "FVORANG", "FVBEANS", "GRENDAY_", "ORNGDAY_", "BEANDAY_", "VEGEDA1_", "_MISVEGN",
    "FRUITJU1", "FRUIT1", "FTJUDA1_", "FRUTDA1_", "_MISFRTN", "_FRTRESP",
    "VEGETAB1", "FVGREEN", "FVORANG", "FVBEANS", "GRENDAY_", "ORNGDAY_", "BEANDAY_", "VEGEDA1_", "_MISVEGN", "_VEGRESP",
    "FRUITJU1", "FRUIT1", "FTJUDA1_", "FRUTDA1_", "_FRUTSUM",
    "VEGETAB1", "FVGREEN", "FVORANG", "FVBEANS", "GRENDAY_", "ORNGDAY_", "BEANDAY_", "VEGEDA1_", "_VEGESUM",
    "FRUITJU1", "FRUIT1", "VEGETAB1", "FVGREEN", "FVORANG", "FVBEANS", "FTJUDA1_", "FRUTDA1_", "VEGEDA1_", "GRENDAY_", "ORNGDAY_", "BEANDAY_", "_FRUTVEG",
    "XPA1MIN_", "PA1MIN_",
    "XPA1MIN_", "_PACAT1",
    "PA1MIN_", "_METVL11",
    "_METVL11", "_METVL21",
    "PAQ650", "_RFPAVIG",
    "PAQ655", "PAQ660", "_PAQ6C",
    "PAQ660", "_PAINDX2",
    "PAQ650", "PAQ665", "_PASTRNG",
    "PAQ670", "_PAREC1"
]
variables_set= set(variables)


In [87]:
data_path="../data/dataset_to_release"
x_train, x_test, y_train, train_ids, test_ids, col_names_train, col_names_test = load_csv_data(data_path, selected_cols=variables_set)

In [90]:
# Check the distribution of the _MICHD labels
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{-1: 299160, 1: 28975}


In [83]:
def build_k_indices_with_seed(y, k_fold, seed=42):
    """
    Randomly partitions the indices of the dataset into k groups using a fixed seed.
    Args:
        y: labels, used for indexing
        k_fold: number of groups after the partitioning
        seed: the random seed value, default is 42
    Returns:
        k_indices: an array of k sub-indices that are randomly partitioned
    """
    num_rows = y.shape[0]
    interval = int(num_rows / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_rows)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [127]:
def calculate_metrics(y_pred,y_true):
    accuracy=y_pred[y_pred==y_true].shape[0]/y_pred.shape[0]
    precision={};recall={}
    for label in np.unique(y_true):
        try:
            precision[label]=y_pred[(y_pred==label) & (y_true==label)].shape[0]/y_pred[y_pred==label].shape[0]
        except:
            continue
        try:
            recall[label]=y_pred[(y_pred==label) & (y_true==label)].shape[0]/y_true[y_true==label].shape[0]
        except:
            continue
    return accuracy,precision,recall

def prepare_data_for_fold(y, x, k_indices, k):
    """
    Prepare the data for a given fold index.

    Args:
        y: Vector of labels
        x: Feature matrix
        k_indices: 2D array returned by build_k_indices()
        k: scalar, the k-th fold
        degree: Degree for polynomial expansion
        high_skewness_cols: Columns that need log transform
        high_residual_indices: Columns that need polynomial expansion

    Returns:
        train_x, train_y, test_x, test_y: Prepared data for the fold
    """
    
    test_x, test_y = x[k_indices[k]], y[k_indices[k]]
    train_x, train_y = (
        x[k_indices[(np.arange(len(k_indices)) != k)].reshape(-1)],
        y[k_indices[(np.arange(len(k_indices)) != k)].reshape(-1)],
    )
    

    
    #print(f"Shape of train_x before preprocessing: {train_x.shape}")
    #print(f"Shape of test_x before preprocessing: {test_x.shape}")
    
    #train_x, train_y = oversample_minority(train_x, train_y)
    train_x, test_x = apply_preprocessing(train_x, test_x)

    #print(f"Shape of train_x after preprocessing: {train_x.shape}")
    #print(f"Shape of test_x after preprocessing: {test_x.shape}")
    
    return train_x, train_y, test_x, test_y

def predict(data, weights, is_logistic):
        preds = data.dot(weights)
        if is_logistic:
            preds = np.where(sigmoid(preds) > 0.5, 1, -1)
        else:
            preds = np.where(preds >= 0, 1, -1)
        return preds

def train_and_evaluate(train_x, train_y, test_x, test_y, model_func, weights=None, 
                       lambda_=0.1, max_iters=1000, gamma=0.01):
    """
    Train a model and evaluate it. Now adapted for weighted training and flexible arguments.

    Args:
        train_x, train_y: Training data
        test_x, test_y: Testing data
        model_func: The regression function to use
        weights: Sample weights
        lambda_: Regularization parameter (if applicable)
        max_iters: Maximum number of iterations for iterative optimization methods, default 1000.
        gamma: Learning rate for optimization methods, default 0.01.
        **kwargs: Additional arguments for model_func

    Returns:
        Training and testing losses, accuracy, precision, and recall
    """

    initial_w = np.zeros(train_x.shape[1])
    #print(f"Shape of train_x: {train_x.shape}")
    #print(f"Shape of initial_w: {initial_w.shape}")
    
    is_logistic = model_func in [logistic_regression, reg_logistic_regression, reg_logistic_regression_weighted, logistic_regression_weighted]
    train_y_adjusted = (train_y + 1) / 2 if is_logistic else train_y
    
    # If weights are not provided, assume equal weights for all samples.
    if weights is None:
        weights = np.ones(train_y.shape[0])
    
    initial_w = np.zeros(train_x.shape[1])
    
    if model_func in [mean_squared_error_gd_weighted, mean_squared_error_sgd_weighted, logistic_regression_weighted]:
        w, _ = model_func(train_y_adjusted, train_x, initial_w, max_iters, gamma, weights)
    elif model_func in [reg_logistic_regression_weighted]:
        w, _ = model_func(train_y_adjusted, train_x, lambda_, initial_w, max_iters, gamma, weights)
    elif model_func in [least_squares_weighted, ridge_regression_weighted]:
        w, _ = model_func(train_y_adjusted, train_x, lambda_, weights)
    elif  model_func in [mean_squared_error_gd, mean_squared_error_sgd, logistic_regression]:
        w, _ = model_func(train_y_adjusted, train_x, initial_w, max_iters, gamma)
    elif model_func in  [reg_logistic_regression]:
        w, _ = model_func(train_y_adjusted, train_x, lambda_, initial_w, max_iters, gamma)
    elif model_func in [least_squares, ridge_regression]:
        w, _ = model_func(train_y_adjusted, train_x, lambda_)
    else:
        raise ValueError("Invalid model selection")
    
    
    preds_train = predict(train_x, w, is_logistic)
    loss_train = np.sqrt(np.mean((train_y - preds_train)** 2))
    acc_train, precision_train, recall_train = calculate_metrics(preds_train, train_y)

    preds_test = predict(test_x, w, is_logistic)
    loss_test = np.sqrt(np.mean((test_y - preds_test) ** 2)) 
    acc_test, precision_test, recall_test = calculate_metrics(preds_test, test_y)

    return loss_train, loss_test, acc_train, acc_test, precision_train, recall_train, precision_test, recall_test


In [128]:
def calculate_f1_score(precision, recall):
    """Compute the F1 score given precision and recall."""
    f1_scores = {}
    for label in precision.keys():
        if precision[label] + recall[label] != 0:
            f1_scores[label] = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label])
        else:
            f1_scores[label] = 0.0
    return f1_scores

def calculate_class_weights(y):
    """Calculate class weights based on inverse class frequencies."""
    class_weights = {}
    unique_classes, counts = np.unique(y, return_counts=True)
    total_samples = y.shape[0]
    for cls, count in zip(unique_classes, counts):
        class_weights[cls] = total_samples / count
        return class_weights

def grid_search_hyperparameters_weighted(y, x, k_indices):
    """Perform grid search with progress tracking."""
    gammas = [0.001, 0.01, 0.1]
    lambdas = [0.001, 0.01, 0.1, 1, 10]
    max_iters_values = [100, 500, 1000, 2000]
    model_funcs = [mean_squared_error_gd, mean_squared_error_sgd, least_squares, ridge_regression, 
                   logistic_regression, reg_logistic_regression,
                   mean_squared_error_gd_weighted, mean_squared_error_sgd_weighted, 
                   least_squares_weighted, ridge_regression_weighted,
                   logistic_regression_weighted, reg_logistic_regression_weighted]
    
    best_accuracy = 0
    best_f1 = 0
    best_hyperparameters = None
    
    total_models = len(model_funcs)
    for idx, model_func in enumerate(model_funcs):
        print(f"Evaluating Model {idx+1}/{total_models}: {model_func.__name__}")
        for gamma in gammas:
            for lambda_ in lambdas:
                for max_iters in max_iters_values:
                    print(f"Using hyperparameters: gamma={gamma}, lambda={lambda_}, max_iters={max_iters}")
                    avg_accuracy = 0
                    avg_f1 = 0
                    for k in range(len(k_indices)):
                        print(f"Processing Fold {k+1}/{len(k_indices)}")
                        train_x, train_y, test_x, test_y = prepare_data_for_fold(y, x, k_indices, k)
                        weights = None
                        if "weighted" in model_func.__name__:
                            class_weights = calculate_class_weights(train_y)
                            weights = np.array([class_weights[label] for label in train_y])
                        _, _, _, acc_test, precision_train, recall_train, precision_test, recall_test = train_and_evaluate(
                            train_x, train_y, test_x, test_y, model_func, weights, lambda_=lambda_, max_iters=max_iters, gamma=gamma)
                        avg_accuracy += acc_test
                        f1 = calculate_f1_score(precision_test, recall_test)
                        avg_f1 += np.mean(list(f1.values()))
                    avg_accuracy /= len(k_indices)
                    avg_f1 /= len(k_indices)
                    print("avg f1:", avg_f1)
                    if avg_accuracy > best_accuracy and avg_f1 > best_f1:
                        best_accuracy = avg_accuracy
                        best_f1 = avg_f1
                        best_hyperparameters = {"model_func": model_func, "gamma": gamma, "lambda": lambda_, "max_iters": max_iters}
                        
    return best_hyperparameters, best_accuracy, best_f1
                 


In [129]:
k_fold = 5  # for 5-fold cross-validation
seed = 42  
k_indices = build_k_indices_with_seed(y_train, k_fold, seed)

best_hyperparameters, best_accuracy, best_f1 = grid_search_hyperparameters_weighted(y_train, x_train, k_indices)
print("Best Hyperparameters:", best_hyperparameters)
print("Best Accuracy:", best_accuracy)
print("Best F1 Score:", best_f1)


Evaluating Model 1/12: mean_squared_error_gd
Using hyperparameters: gamma=0.001, lambda=0.001, max_iters=100
Processing Fold 1/5


c:\Users\zeyne\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\zeyne\OneDrive\Masaüstü\project1-ML\Turco-ML\project1\trials\helpers_trials.py:19: RuntimeWarning: overflow encountered in square
  return np.mean((y - y_pred) ** 2) / 2
c:\Users\zeyne\OneDrive\Masaüstü\project1-ML\Turco-ML\project1\trials\implementations_trial.py:22: RuntimeWarning: invalid value encountered in subtract
  w = w - gamma * gradient_vector


Processing Fold 2/5
Processing Fold 3/5
Processing Fold 4/5
Processing Fold 5/5
avg f1: 0.9538092018066905
Using hyperparameters: gamma=0.001, lambda=0.001, max_iters=500
Processing Fold 1/5
Processing Fold 2/5
Processing Fold 3/5
Processing Fold 4/5
Processing Fold 5/5
avg f1: 0.9538092018066905
Using hyperparameters: gamma=0.001, lambda=0.001, max_iters=1000
Processing Fold 1/5


KeyboardInterrupt: 